# Data Cleaning 

### From Kaggle Kernels:
#### Extensive Sberbank Exploratory Analysis:
**source:** https://www.kaggle.com/captcalculator/a-very-extensive-sberbank-exploratory-analysis/comments#latest-185178

* 37 observations where life_sq is greater than full_sq.
* A vast majority of the apartments have three rooms or less.
* Look to see how life sq ~ price changes based on sub-area or distance to kremlin
* Home price does seem to increase with population density.
* There does not appear to be a relationship between the mean home price in a district and the district’s share of working age population.
* Surprisingly, there is little to no correlation between price and the school variables. The school variables however are highly correlated with each other, indicating that we would not want to use all of them in a linear regression model due to multicollinearity.
* homes with >3 top 20 universities show signs of correlation, but only one house fits that description.
* raions that have a top 25 cultural object have a median home sale price that is higher by 1.2 million (using this feature as a factor)
* strong positive correlation between sport_objects_raion and price_doc

#### Simple Exploration Notebook - Sberbank
**source:** https://www.kaggle.com/sudalairajkumar/simple-exploration-notebook-sberbank/comments/notebook

* overall increasing trend in price as floor_num increases. A sudden increase in the house price is also observed at floor 18.
* Individual houses seems to be costlier, check price of 0 floor houses.

#### Map visualizations with external shapefile
**source:** https://www.kaggle.com/jtremoureux/map-visualizations-with-external-shapefile/notebook

* replace regions with map shapefile coordinates



## Import Libraries & Data

In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from datetime import datetime

# load dataset
train_df = pd.read_csv("../Sberbank/train.csv", parse_dates=['timestamp'], index_col=False)
test_df = pd.read_csv("../Sberbank/test.csv", parse_dates=['timestamp'], index_col=False)
macro_df = pd.read_csv("../Sberbank/macro.csv", parse_dates=['timestamp'], index_col=False)

### Merge Train & Test

In [2]:
frames = [train_df, test_df]

df = pd.concat(frames)

### Add Macro

In [3]:
# add macro data to train/test data

train_df = pd.merge(train_df, macro_df, how='left', on='timestamp')
test_df = pd.merge(test_df, macro_df, how='left', on='timestamp')

df = pd.merge(df, macro_df, how='left', on='timestamp')

## Transformations

### Add Date/Time 

In [4]:
# add month, day, year to train and test data
df['month'] = df['timestamp'].dt.month
df['day'] = df['timestamp'].dt.day
df['year'] = df['timestamp'].dt.year

# Add month-year
month_year = (df.timestamp.dt.month + df.timestamp.dt.year * 100)
month_year_map = month_year.value_counts().to_dict()
df['month_year'] = month_year.map(month_year_map)


# Add week-year count
week_year = (df.timestamp.dt.weekofyear + df.timestamp.dt.year * 100)
week_year_map = week_year.value_counts().to_dict()
df['week_year'] = week_year.map(week_year_map)


# Add month and day-of-week
df['month'] = df.timestamp.dt.month
df['dow'] = df.timestamp.dt.dayofweek

### Price:

In [5]:
df['price_doc_log'] = np.log1p(df['price_doc'])
df['price_doc_log10'] = np.log10(df['price_doc'])

### Unit Size:

In [6]:
# squared and square root of full sq
df['full_sq^2'] = np.square(df['full_sq'])
df['full_sqrt'] = np.sqrt(df['full_sq'])

# floor ratios
df['rel_floor'] = df['floor'] / df['max_floor'].astype(float)
df['rel_kitch_sq'] = df['kitch_sq'] / df['full_sq'].astype(float)


# area and population density
df['area_km'] = df['area_m'] / 1000000
df['density'] = df['raion_popul'] / df['area_km']


# working population
df['work_share'] = df['work_all'] / df['raion_popul']

# Adding Lags and calculating deltas

In [ ]:
# Create lag timestamps for one month, three months, 6 months, and 12 months in df
# Join with appropriate macro

mini_macro.columns = ['one_month_lag','oil_urals_1', 'gdp_quart_1', 'cpi_1', 
                      'usdrub_1', 'micex_cbi_tr_1', 'labor_force_1']
df['one_month_lag'] = df['timestamp'] - pd.to_timedelta('30 days')
df = pd.merge(df, mini_macro, how = 'left', on = 'one_month_lag')

# 3 month lage
mini_macro.columns = ['three_month_lag','oil_urals_3', 'gdp_quart_3', 'cpi_3', 
                      'usdrub_3', 'micex_cbi_tr_3', 'labor_force_3']
df['three_month_lag'] = df['timestamp'] - pd.to_timedelta('90 days')
df = pd.merge(df, mini_macro, how = 'left', on = 'three_month_lag')

# 6 month lage
mini_macro.columns = ['six_month_lag','oil_urals_6', 'gdp_quart_6', 'cpi_6', 
                      'usdrub_6', 'micex_cbi_tr_6', 'labor_force_6']
df['six_month_lag'] = df['timestamp'] - pd.to_timedelta('180 days')
df = pd.merge(df, mini_macro, how = 'left', on = 'six_month_lag')

# 12 month lage
mini_macro.columns = ['twelve_month_lag','oil_urals_12', 'gdp_quart_12', 'cpi_12', 
                      'usdrub_12', 'micex_cbi_tr_12', 'labor_force_12']
df['twelve_month_lag'] = df['timestamp'] - pd.to_timedelta('360 days')
df = pd.merge(df, mini_macro, how = 'left', on = 'twelve_month_lag')

In [ ]:
# calculate the deltas
df['delta_oil_1'] = (df['oil_urals'] - df['oil_urals_1']) / df['oil_urals']
df['delta_oil_3'] = (df['oil_urals'] - df['oil_urals_3']) / df['oil_urals']
df['delta_oil_6'] = (df['oil_urals'] - df['oil_urals_6']) / df['oil_urals']
df['delta_oil_12'] = (df['oil_urals'] - df['oil_urals_12']) / df['oil_urals']

df['delta_usdrub_1'] = (df['usdrub'] - df['usdrub_1']) / df['usdrub']
df['delta_usdrub_3'] = (df['usdrub'] - df['usdrub_3']) / df['usdrub']
df['delta_usdrub_6'] = (df['usdrub'] - df['usdrub_6']) / df['usdrub']
df['delta_usdrub_12'] = (df['usdrub'] - df['usdrub_12']) / df['usdrub']

df['delta_labor_force_1'] = (df['labor_force'] - df['labor_force_1']) / df['labor_force']
df['delta_labor_force_3'] = (df['labor_force'] - df['labor_force_3']) / df['labor_force']
df['delta_labor_force_6'] = (df['labor_force'] - df['labor_force_6']) / df['labor_force']
df['delta_labor_force_12'] = (df['labor_force'] - df['labor_force_12']) / df['labor_force']

### Manual Imputing:

In [7]:
# Cleaning values with incorrect information 
train_df.loc[train_df['state'] == 33, 'status'] = 3
train_df = train_df.loc[train_df['id'] != 24718] 
train_df = train_df.loc[train_df['floor'] != 0] 

train_df.loc[train_df['build_year'] == 20052009, 'build_year'] = 2009
train_df.loc[train_df['build_year'] == 215, 'build_year'] = 2015
train_df = train_df.loc[train_df['build_year'] != 4965, :]
df.loc[df['build_year'] == 0, 'build_year'] = 2013
df.loc[df['build_year'] == 1, 'build_year'] = 2014
df.loc[df['build_year'] == 2, 'build_year'] = 2015

## Functions for Scaling/Normalizing/Reshaping
 (Note: not necessary for Random Forest Models)

In [8]:
def rescale(feature):
    return feature.apply(lambda x: (x - np.mean(x))/np.std(x))

def normalize(feature):
    return feature.apply(lambda x: (x-np.min(x))/(np.max(x)-np.min(x)))

#### Check dtypes and split objects and numeric types

In [9]:
df_numerics = df.select_dtypes(exclude=['object'])
df_objects = df.select_dtypes(include=['object'])

#### Normalize and rescale numeric features

In [10]:
df_numerics.loc[:, df_numerics.columns != 'timestamp'] = normalize(rescale(df_numerics.select_dtypes(exclude=['datetime'])))

/Users/rezarad/anaconda2/lib/python2.7/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


#### Convert Categorical Variables to Numerics and merge again

In [11]:
frames = [df_numerics, df_objects]

df = pd.concat(frames,axis=1)

In [12]:
df.shape

(38133, 406)

### Creating X and y sets

In [13]:
y_full = df['price_doc'].values
X_full = df.loc[:, df.columns != 'price_doc'].values

Check to see if any feature consists of non-float value

## Save to CSV

In [14]:
df.to_csv('cleaned_df_{0}.csv'.format(datetime.date(datetime.now())), index=False)